# PART- 2 `( Handling Missing Values )`

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats.mstats import winsorize

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

%matplotlib inline
# %matplotlib notebook

plt.rcParams["figure.figsize"] = (10,6)
# plt.rcParams['figure.dpi'] = 100

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.3f' % x)

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 150

In [2]:
df = pd.read_csv("clean_scout.csv")

In [3]:
df.shape

(15919, 33)

In [4]:
df.head(3).T

,0,1,2
make_model,Audi A1,Audi A1,Audi A1
body_type,Sedans,Sedans,Sedans
price,15770,14500,14640
vat,VAT deductible,Price negotiable,VAT deductible
km,56013.000,80000.000,83450.000
Type,Used,Used,Used
Warranty,NaN,NaN,NaN
Cylinders,3.000,4.000,NaN
Fuel,Diesel,Benzine,Diesel
Gears,NaN,7.000,NaN


In [5]:
df.isnull().sum()/df.shape[0]*100

make_model             0.000
body_type              0.377
price                  0.000
vat                   28.350
km                     6.433
Type                   0.013
Warranty              69.514
Cylinders             35.681
Fuel                   0.000
Gears                 29.600
Comfort_Convenience    5.779
Entertainment_Media    8.631
Extras                18.607
Safety_Security        6.169
age                   10.032
Previous_Owners       41.711
hp_kW                  0.553
Inspection_new        75.300
Body_Color             3.750
Paint_Type            36.259
Upholstery_type       30.599
Upholstery_color      31.899
Nr_of_Doors            1.332
Nr_of_Seats            6.137
Gearing_Type           0.000
Displacement_cc        3.116
Weight_kg             43.809
Drive_chain           43.081
cons_comb             12.771
cons_city             15.302
cons_country          14.926
CO2_Emission          15.302
Emission_Class        22.790
dtype: float64

In [6]:
miss_val = []

[miss_val.append(i) for i in df.columns if any(df[i].isnull())]

miss_val

['body_type',
 'vat',
 'km',
 'Type',
 'Warranty',
 'Cylinders',
 'Gears',
 'Comfort_Convenience',
 'Entertainment_Media',
 'Extras',
 'Safety_Security',
 'age',
 'Previous_Owners',
 'hp_kW',
 'Inspection_new',
 'Body_Color',
 'Paint_Type',
 'Upholstery_type',
 'Upholstery_color',
 'Nr_of_Doors',
 'Nr_of_Seats',
 'Displacement_cc',
 'Weight_kg',
 'Drive_chain',
 'cons_comb',
 'cons_city',
 'cons_country',
 'CO2_Emission',
 'Emission_Class']

In [7]:
# function for first looking to the columns

def first_looking(col):
    print("column name    : ", col)
    print("--------------------------------")
    print("per_of_nulls   : ", "%", round(df[col].isnull().sum()/df.shape[0]*100, 2))
    print("num_of_nulls   : ", df[col].isnull().sum())
    print("num_of_uniques : ", df[col].nunique())
    print(df[col].value_counts(dropna = False))

## functions to fill the missing values

In [8]:
def fill_most(df, group_col, col_name):
    '''Fills the missing values with the most existing value (mode) in the relevant column according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [9]:
def fill_prop(df, group_col, col_name):
    '''Fills the missing values with "ffill and bfill method" according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [10]:
def fill(df, group_col1, group_col2, col_name, method): # method can be "mode" or "median" or "ffill"
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if mode2 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
                elif mode1 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
                else:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])
                
    elif method == "median":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].median()).fillna(df[cond1][col_name].median()).fillna(df[col_name].median())
                
    elif method == "ffill":           
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
        for group1 in list(df[group_col1].unique()):
            cond1 = df[group_col1]==group1
            df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
        df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

## Let's examine and fill the missing values of all the columns/features one by one

## age

In [11]:
first_looking("age")

column name    :  age
--------------------------------
per_of_nulls   :  % 10.03
num_of_nulls   :  1597
num_of_uniques :  4
1.000    4522
3.000    3674
2.000    3273
0.000    2853
nan      1597
Name: age, dtype: int64


In [12]:
df['age'].fillna('-', inplace=True)

In [13]:
df["age"].value_counts(dropna=False)

1.0    4522
3.0    3674
2.0    3273
0.0    2853
-      1597
Name: age, dtype: int64

In [14]:
df.groupby("age").km.describe()

,count,mean,std,min,25%,50%,75%,max
age,,,,,,,,
0.0,2706.000,2085.355,5365.881,1.000,10.000,50.000,3000.000,127022.000
1.0,4484.000,18035.239,11052.524,1.000,9990.000,17872.000,25078.500,136000.000
2.0,3272.000,41754.941,28295.748,1.000,21541.750,34752.000,54805.500,317000.000
3.0,3674.000,77442.521,39170.143,10.000,48000.000,72914.500,99950.000,291800.000
-,759.000,934.497,7416.244,0.000,5.000,10.000,10.000,89982.000


In [15]:
df[df["age"]=="-"]["km"].value_counts(dropna=False)

nan          838
10.000       369
1.000        146
5.000         58
20.000        32
15.000        21
0.000         19
11.000        12
8.000         11
50.000        10
12.000         8
100.000        8
7.000          7
9.000          4
3.000          4
4.000          3
30.000         3
25.000         3
250.000        3
3000.000       2
39962.000      2
2.000          2
22627.000      2
60.000         1
6.000          1
784.000        1
196.000        1
40.000         1
6100.000       1
85000.000      1
281.000        1
11200.000      1
11000.000      1
19500.000      1
325.000        1
5000.000       1
81800.000      1
68485.000      1
99.000         1
77.000         1
500.000        1
150.000        1
142.000        1
20768.000      1
89982.000      1
89692.000      1
32084.000      1
497.000        1
141.000        1
4500.000       1
3500.000       1
34164.000      1
4307.000       1
Name: km, dtype: int64

In [16]:
cond1 = (df['km'] < 10000)
cond2 = ((df['km'] >= 10000) & (df['km'] < 28000))
cond3 = ((df['km'] >= 28000) & (df['km'] < 50000))
cond4 = (df['km'] >= 50000)

In [17]:
df.loc[cond1,'age'] = df.loc[cond1,'age'].replace('-', 0)
df.loc[cond2,'age'] = df.loc[cond2,'age'].replace('-', 1)
df.loc[cond3,'age'] = df.loc[cond3,'age'].replace('-', 2)
df.loc[cond4,'age'] = df.loc[cond4,'age'].replace('-', 3)

In [18]:
df.groupby('age').km.mean()

age
0.0    1647.363
1.0   18035.130
2.0   41748.577
3.0   77450.063
-           nan
Name: km, dtype: float64

In [19]:
df["age"].value_counts(dropna=False)

1.0    4528
3.0    3679
0.0    3597
2.0    3277
-       838
Name: age, dtype: int64

In [20]:
df.groupby(['make_model', 'age']).km.describe()

count      mean       std       min       25%       50%  \
make_model     age                                                             
Audi A1        0.0 569.000  2103.459  3258.850     0.000    10.000   100.000   
               1.0 744.000 13806.144  7975.693     1.000  7466.750 12413.000   
               2.0 432.000 25821.713 18608.364    10.000 14252.500 20730.500   
               3.0 629.000 54332.286 26281.269  3150.000 34914.000 50000.000   
               -     0.000       nan       nan       nan       nan       nan   
Audi A2        1.0   1.000 26166.000       nan 26166.000 26166.000 26166.000   
Audi A3        0.0 671.000  1517.817  6548.643     0.000    10.000    10.000   
               1.0 776.000 18410.524 11054.420     1.000 11200.000 18000.000   
               2.0 675.000 43853.141 27349.704    15.000 25000.000 36677.000   
               3.0 818.000 90092.983 36464.156    35.000 62863.250 88000.000   
               -     0.000       nan       nan       nan       nan       nan   
Opel Astra     0.0 541.000  2351.275  5103.476     0.000    10.000    50.000   
               1.0 721.000 21442.709 10900.190     5.000 14000.000 22000.000   
               2.0 587.000 49981.779 31163.248    30.000 25201.500 44179.000   
               3.0 521.000 89086.697 42021.315 10027.000 58021.000 84900.000   
               -     0.000       nan       nan       nan       nan       nan   
Opel Corsa     0.0 647.000   913.544  3591.394     0.000    10.000    10.000   
               1.0 684.000 17431.599 11739.606     5.000  8381.250 16617.000   
               2.0 386.000 39942.383 25855.345    99.000 21075.000 39905.500   
               3.0 430.000 62947.974 36418.878  1800.000 34803.750 55484.000   
               -     0.000       nan       nan       nan       nan       nan   
Opel Insignia  0.0 435.000  2419.908  4165.213     0.000    10.000    77.000   
               1.0 767.000 22179.207  8486.335     1.000 16217.500 23928.000   
               2.0 589.000 47732.104 32834.328     1.000 22561.000 33900.000   
               3.0 568.000 93182.342 41020.814 10791.000 63000.000 88808.500   
               -     0.000       nan       nan       nan       nan       nan   
Renault Clio   0.0 332.000   581.717  5865.375     0.000     9.750    10.000   
               1.0 564.000 15396.518 13314.804     1.000  5296.250 13685.000   
               2.0 437.000 36660.153 21098.396    20.000 23000.000 32713.000   
               3.0 413.000 68615.436 35580.167    10.000 42208.000 63404.000   
               -     0.000       nan       nan       nan       nan       nan   
Renault Duster 0.0  16.000   101.000     0.000   101.000   101.000   101.000   
               -     0.000       nan       nan       nan       nan       nan   
Renault Espace 0.0 239.000  1496.105  3501.074     0.000    10.000    15.000   
               1.0 233.000 14226.330 11099.269     1.000  5250.000 13625.000   
               2.0 170.000 41886.635 27216.915     1.000 22313.750 41115.500   
               3.0 300.000 74400.567 34189.093  3532.000 49975.000 68766.000   
               -     0.000       nan       nan       nan       nan       nan   

                          75%        max  
make_model     age                        
Audi A1        0.0   3333.000  18000.000  
               1.0  20309.750  47000.000  
               2.0  32028.750 148257.000  
               3.0  65500.000 192000.000  
               -          nan        nan  
Audi A2        1.0  26166.000  26166.000  
Audi A3        0.0    100.000 127022.000  
               1.0  24215.000 136000.000  
               2.0  55251.000 158000.000  
               3.0 112562.500 291800.000  
               -          nan        nan  
Opel Astra     0.0   3698.000  88000.000  
               1.0  28466.000  67500.000  
               2.0  69188.000 188000.000  
               3.0 112388.000 260000.000  
               -          nan        nan  
Opel Corsa     0.0    100.000  76300.000  
   

In [21]:
df.groupby(['make_model',"body_type", 'age']).price.describe()

count      mean       std       min  \
make_model     body_type     age                                         
Audi A1        Compact       0.0 198.000 23277.434  3510.406 14900.000   
                             1.0 268.000 18596.041  2659.910 13980.000   
                             2.0 161.000 16602.807  2085.384 10999.000   
                             3.0 234.000 14532.910  1908.909  9950.000   
                             -   178.000 23996.264  3383.852 16220.000   
               Coupe         2.0   1.000 15900.000       nan 15900.000   
                             3.0   1.000 13950.000       nan 13950.000   
               Other         0.0   8.000 23826.250  2057.439 21490.000   
                             1.0   3.000 16796.667   178.979 16590.000   
                             2.0   1.000 23490.000       nan 23490.000   
                             -     1.000 21850.000       nan 21850.000   
               Sedans        0.0 368.000 24158.087  3196.985 15990.000   
                             1.0 465.000 18394.938  2588.159 13450.000   
                             2.0 266.000 16605.395  2223.979 11600.000   
                             3.0 389.000 14466.900  1890.278  8999.000   
                             -    50.000 23336.560  3815.326 15980.000   
               Station wagon 0.0   2.000 28944.500    77.075 28890.000   
                             1.0  11.000 17410.909  2166.031 15750.000   
                             2.0   3.000 17613.333  2953.817 14750.000   
                             3.0   5.000 14358.000  1276.891 12950.000   
               Van           0.0   1.000 29000.000       nan 29000.000   
Audi A2        Off-Road      1.0   1.000 28200.000       nan 28200.000   
Audi A3        Compact       0.0  17.000 29345.471  8767.065 15500.000   
                             1.0  29.000 22030.931  4270.469 14500.000   
                             2.0  39.000 20489.769  8208.445 13790.000   
                             3.0  93.000 15303.796  2256.723 12900.000   
                             -     4.000 25582.500  1185.109 24542.000   
               Convertible   1.0   3.000 29664.333 23557.901 10893.000   
                             2.0   4.000 28562.500  5301.159 20900.000   
                             3.0   1.000 20000.000       nan 20000.000   
               Coupe         2.0   1.000 19000.000       nan 19000.000   
                             3.0   3.000 17326.667   583.124 16990.000   
               Other         1.0   4.000 23422.500  3548.055 20200.000   
                             2.0   4.000 18675.000  1351.851 17650.000   
                             3.0   5.000 15970.000  2392.070 12550.000   
                             -     3.000 25990.000  1322.876 24490.000   
               Sedans        0.0 626.000 24909.898  2354.990 18820.000   
                             1.0 709.000 22544.715  5224.410 14250.000   
                             2.0 550.000 19980.358  3721.376 10550.000   
                             3.0 647.000 16702.832  2900.228 10950.000   
                             -    66.000 24395.242  2372.569 19750.000   
               Station wagon 0.0  69.000 25219.000  5011.257 17990.000   
                             1.0  32.000 21252.594  3201.415  5555.000   
                             2.0  72.000 19010.875  1522.622 14590.000   
                             3.0  67.000 15739.507  1883.567 11790.000   
                             -    42.000 23558.667  1687.726 18980.000   
Opel Astra     Compact       0.0  51.000 23056.118  7391.480 16500.000   
                             1.0  39.000 14717.538  1266.385 11999.000   
                             2.0  55.000 13091.491  1884.243  9999.000   
                             3.0  30.000 12023.733  2767.419  8450.000   
                             -    10.000 16636.600  1098.445 15816.000   
               Coupe         0.0   1.000 29990.000       nan 29990.000   
                             3.0   1.00

In [22]:
df['age'].replace('-',0, inplace=True)

In [23]:
df.groupby('age').km.mean()

age
0.000    1647.363
1.000   18035.130
2.000   41748.577
3.000   77450.063
Name: km, dtype: float64

In [24]:
df["age"].value_counts(dropna=False)

1.000    4528
0.000    4435
3.000    3679
2.000    3277
Name: age, dtype: int64

## km

In [25]:
first_looking("km")

column name    :  km
--------------------------------
per_of_nulls   :  % 6.43
num_of_nulls   :  1024
num_of_uniques :  6689
10.000       1045
nan          1024
1.000         367
5.000         170
50.000        148
             ... 
8329.000        1
267.000         1
1060.000        1
73652.000       1
49320.000       1
Name: km, Length: 6690, dtype: int64


In [26]:
df.groupby("age").km.mean()

age
0.000    1647.363
1.000   18035.130
2.000   41748.577
3.000   77450.063
Name: km, dtype: float64

In [27]:
df.groupby("age").km.transform("mean")

0       77450.063
1       41748.577
2       77450.063
3       77450.063
4       77450.063
           ...   
15914    1647.363
15915    1647.363
15916    1647.363
15917    1647.363
15918    1647.363
Name: km, Length: 15919, dtype: float64

In [28]:
df["km"].fillna(df.groupby("age").km.transform("mean"), inplace=True)

In [29]:
df.km.value_counts(dropna=False)

10.000       1045
1647.363      985
1.000         367
5.000         170
50.000        148
             ... 
79930.000       1
267.000         1
8329.000        1
1060.000        1
15577.000       1
Name: km, Length: 6692, dtype: int64

## body type

In [30]:
first_looking("body_type")

column name    :  body_type
--------------------------------
per_of_nulls   :  % 0.38
num_of_nulls   :  60
num_of_uniques :  9
Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64


In [31]:
df.body_type.replace("Other", np.nan, inplace=True) 

In [32]:
df['body_type'].value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
NaN               350
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [33]:
df["body_type"].mode()

0    Sedans
dtype: object

In [34]:
df["body_type"].mode()[0]

'Sedans'

In [35]:
#Step-1
#df["body_type"].fillna(df["body_type"].mode()[0])

#Step-2
#df.loc[df["make_model"]=="Audi A1", "body_type"].fillna(df[df["make_model"]=="Audi A1"]["body_type"].mode()[0])

In [36]:
for group in list(df["make_model"].unique()):
    cond = df["make_model"]==group
    mode = list(df[cond]["body_type"].mode())
    if mode != []:
        df.loc[cond, "body_type"] = df.loc[cond, "body_type"].fillna(df[cond]["body_type"].mode()[0])
    else:
        df.loc[cond, "body_type"] = df.loc[cond, "body_type"].fillna(df["body_type"].mode()[0])

In [37]:
df['body_type'].value_counts(dropna=False)

Sedans           8005
Station wagon    3678
Compact          3242
Van               817
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [38]:
def fill_most(df, group_col, col_name):
    '''Fills the missing values with the most existing value (mode) in the relevant column according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [39]:
fill_most(df, "make_model", "body_type")

Number of NaN :  0
------------------
Sedans           8005
Station wagon    3678
Compact          3242
Van               817
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64


## Previous_Owners

In [40]:
first_looking("Previous_Owners")

column name    :  Previous_Owners
--------------------------------
per_of_nulls   :  % 41.71
num_of_nulls   :  6640
num_of_uniques :  5
1.000    8294
nan      6640
2.000     778
0.000     188
3.000      17
4.000       2
Name: Previous_Owners, dtype: int64


In [41]:
df["Previous_Owners"].fillna("-", inplace = True)

In [42]:
df["Previous_Owners"].value_counts(dropna=False)

1.0    8294
-      6640
2.0     778
0.0     188
3.0      17
4.0       2
Name: Previous_Owners, dtype: int64

In [43]:
df.groupby(['make_model', 'age', 'Previous_Owners']).km.describe()

count       mean       std       min  \
make_model     age   Previous_Owners                                          
Audi A1        0.000 0.0              45.000    958.365   815.362     0.000   
                     1.0             239.000   3069.543  3460.402     1.000   
                     2.0               1.000   3000.000       nan  3000.000   
                     -               521.000   1549.992  2303.800     1.000   
               1.000 0.0               1.000  15000.000       nan 15000.000   
                     1.0             496.000  13772.192  8099.461    20.000   
                     2.0              14.000  13734.286 10613.039  5000.000   
                     -               236.000  13930.462  7544.057     1.000   
               2.000 0.0               1.000  68002.000       nan 68002.000   
                     1.0             195.000  23858.369 19666.376    10.000   
                     2.0              74.000  20918.946 14313.356  3900.000   
                     3.0               1.000  18890.000       nan 18890.000   
                     4.0               1.000  19700.000       nan 19700.000   
                     -               160.000  30300.025 18023.499   200.000   
               3.000 0.0               4.000  58176.750  5569.647 50707.000   
                     1.0             401.000  54041.192 27128.754  3150.000   
                     2.0              82.000  49948.817 18268.135  4381.000   
                     3.0               1.000  45000.000       nan 45000.000   
                     -               141.000  57666.525 27893.546  8900.000   
Audi A2        1.000 1.0               1.000  26166.000       nan 26166.000   
Audi A3        0.000 0.0              49.000    880.641   823.805     0.000   
                     1.0             257.000   1953.462  3715.728     1.000   
                     -               521.000   1401.636  6946.583     1.000   
               1.000 0.0               5.000  28967.000  8648.989 16749.000   
                     1.0             380.000  17260.627 10710.879     1.000   
                     2.0              11.000  17966.818  6209.465  7900.000   
                     -               381.000  19430.693 11365.016     1.000   
               2.000 1.0             351.000  47381.068 30979.305  3277.000   
                     2.0              47.000  28973.489 19029.482    15.000   
                     -               277.000  41907.440 22179.630    59.000   
               3.000 0.0               1.000  64500.000       nan 64500.000   
                     1.0             458.000  94950.721 38584.468 14000.000   
                     2.0              53.000  67559.189 27702.965 17000.000   
                     3.0               2.000  71800.000 62508.239 27600.000   
                     -               304.000  86907.543 32497.991    35.000   
Opel Astra     0.000 0.0              12.000    555.871   806.230     0.000   
                     1.0             253.000   3601.621  6588.867     2.000   
                     2.0               1.000   7700.000       nan  7700.000   
                     -               425.000   1396.621  2349.957     1.000   
               1.000 0.0               1.000  30800.000       nan 30800.000   
                     1.0             531.000  21259.041  9582.281    10.000   
                     2.0              24.000  19963.458  9864.030   800.000   
                     -               171.000  22046.373 14244.300     5.000   
               2.000 1.0             354.000  52083.944 34251.398  1300.000   
                     2.0              54.000  35318.259 22641.366 12950.000   
                     3.0               1.000  62000.000       nan 62000.000   
                     -               178.000  50182.034 25430.913    30.000   
               3.000 1.0             349.000  95006.731 41188.743 10700.000   
                     2.0              55.000  69682.436 41233.912 10027.000   
          

In [44]:
df[(df["make_model"]=="Renault Duster") & (df["Previous_Owners"] == "-")]["km"]

14894   1647.363
14895   1647.363
14896    101.000
14897   1647.363
14898    101.000
14899    101.000
14900   1647.363
14901    101.000
14903    101.000
14904   1647.363
14905   1647.363
14906    101.000
14907    101.000
14908   1647.363
14909   1647.363
14910    101.000
14911   1647.363
14912    101.000
14913   1647.363
14914    101.000
14915    101.000
14916    101.000
14917   1647.363
14918   1647.363
14919   1647.363
14920    101.000
14921   1647.363
14922   1647.363
14923    101.000
14924   1647.363
14925    101.000
14926    101.000
14927   1647.363
Name: km, dtype: float64

In [45]:
cond = (df["make_model"]=="Renault Duster") & (df["Previous_Owners"] == "-")
df.loc[cond, "Previous_Owners"] = df.loc[cond, "Previous_Owners"].replace("-", 0.0)

In [46]:
df["Previous_Owners"].value_counts(dropna=False)

1.0    8294
-      6607
2.0     778
0.0     221
3.0      17
4.0       2
Name: Previous_Owners, dtype: int64

In [47]:
df["Previous_Owners"].replace("-", np.nan, inplace=True)

In [48]:
#Step-1
#df.["Previous_Owners"].fillna(method="ffill")

#Step-2
#df.loc[df["age"]==0, "Previous_Owners"].fillna(method="ffill")

In [49]:
#Step-3
for group in list(df["age"].unique()):
    cond = df["age"]==group
    df.loc[cond, "Previous_Owners"] = df.loc[cond, "Previous_Owners"].fillna(method="ffill").fillna(method="bfill")
df["Previous_Owners"] = df["Previous_Owners"].fillna(method="ffill").fillna(method="bfill")

In [50]:
df["Previous_Owners"].value_counts(dropna=False)

1.000    14162
2.000     1172
0.000      554
3.000       29
4.000        2
Name: Previous_Owners, dtype: int64

In [51]:
def fill_prop(df, group_col, col_name):
    '''Fills the missing values with "ffill and bfill method" according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [52]:
fill_prop(df, "age", "Previous_Owners")

Number of NaN :  0
------------------
1.000    14162
2.000     1172
0.000      554
3.000       29
4.000        2
Name: Previous_Owners, dtype: int64


## Warranty

In [53]:
first_looking("Warranty")

column name    :  Warranty
--------------------------------
per_of_nulls   :  % 69.51
num_of_nulls   :  11066
num_of_uniques :  41
nan       11066
12.000     2594
24.000     1118
60.000      401
36.000      279
48.000      149
6.000       125
72.000       59
3.000        33
23.000       11
18.000       10
20.000        7
25.000        6
2.000         5
26.000        4
50.000        4
16.000        4
1.000         3
4.000         3
34.000        3
19.000        3
13.000        3
11.000        2
21.000        2
14.000        2
17.000        2
45.000        2
46.000        2
9.000         2
22.000        2
28.000        2
56.000        1
47.000        1
10.000        1
8.000         1
7.000         1
40.000        1
33.000        1
65.000        1
15.000        1
30.000        1
49.000        1
Name: Warranty, dtype: int64


In [54]:
df["Warranty"].fillna("-", inplace = True)

In [55]:
df.groupby(['make_model', 'age', 'Warranty']).price.describe()

count      mean      std       min       25%  \
make_model     age   Warranty                                                  
Audi A1        0.000 12.0      38.000 23846.342 2978.850 17900.000 21000.000   
                     18.0       1.000 19995.000      nan 19995.000 19995.000   
                     24.0     107.000 23766.308 3338.106 14900.000 21890.000   
                     36.0      26.000 25514.423 3205.836 19900.000 22489.250   
                     48.0      18.000 24441.667 2516.782 19650.000 22800.000   
                     56.0       1.000 21760.000      nan 21760.000 21760.000   
                     60.0      14.000 21829.857  717.879 20990.000 21340.000   
                     -        601.000 23854.852 3442.695 15980.000 21390.000   
               1.000 6.0        9.000 22166.667  743.303 20900.000 22400.000   
                     12.0      64.000 18106.406 2408.814 14220.000 16429.750   
                     13.0       1.000 19840.000      nan 19840.000 19840.000   
                     20.0       1.000 17950.000      nan 17950.000 17950.000   
                     23.0       1.000 20950.000      nan 20950.000 20950.000   
                     24.0      88.000 18378.455 2614.704 14500.000 16437.500   
                     26.0       1.000 15840.000      nan 15840.000 15840.000   
                     36.0      26.000 18011.692 2314.022 15500.000 15980.000   
                     46.0       1.000 15980.000      nan 15980.000 15980.000   
                     47.0       1.000 16370.000      nan 16370.000 16370.000   
                     48.0      26.000 17903.077 2068.172 14900.000 15900.000   
                     49.0       1.000 16270.000      nan 16270.000 16270.000   
                     50.0       3.000 17903.333 1342.882 16370.000 17420.000   
                     60.0      11.000 16426.000  434.340 15900.000 15900.000   
                     -        513.000 18542.497 2664.307 13450.000 16445.000   
               2.000 3.0        2.000 20740.000 3889.087 17990.000 19365.000   
                     6.0        3.000 18443.333 2475.284 15990.000 17195.000   
                     12.0      80.000 16187.125 1918.723 13500.000 14950.000   
                     24.0      33.000 16858.455 2723.500 11990.000 14990.000   
                     36.0       8.000 17129.250 2677.735 14400.000 14880.000   
                     48.0       4.000 19800.000  115.470 19700.000 19700.000   
                     60.0       1.000 14990.000      nan 14990.000 14990.000   
                     -        301.000 16621.209 2131.491 10999.000 15280.000   
               3.000 3.0        2.000 13235.000  346.482 12990.000 13112.500   
                     6.0        9.000 15632.000 2329.396 12980.000 13990.000   
                     12.0     120.000 14324.950 1914.102  9950.000 12990.000   
                     23.0       1.000 12170.000      nan 12170.000 12170.000   
                     24.0      13.000 14540.000 1970.787 12500.000 13000.000   
                     60.0       4.000 13467.250  391.157 12999.000 13217.250   
                     -        480.000 14526.779 1877.216  8999.000 13400.000   
Audi A2        1.000 -          1.000 28200.000      nan 28200.000 28200.000   
Audi A3        0.000 3.0        1.000 17990.000      nan 17990.000 17990.000   
                     11.0       1.000 26795.000      nan 26795.000 26795.000   
                     12.0      46.000 24362.783 1527.389 21900.000 22900.000   
                     19.0       2.000 26273.000  388.909 25998.000 26135.500   
                     24.0     104.000 25156.144 4333.166 20290.000 23900.000   
                     36.0      73.000 25524.781 1657.660 20000.000 24300.000   
                     48.0      19.000 24593.684 1326.973 21900.000 24100.000   
                     60.0       4.000 23942.500 2531.737 20990.000 22490.000   
                     72.0       1.000 23990.000      nan 23990.000 23990.000   
                     -        57

##### There are too many nan values and when we analyzed these nan values according to the km, age and make_model columns, we decided that this column does not have healthy data.

In [56]:
df.drop("Warranty", axis=1, inplace=True)

## vat

In [57]:
first_looking("vat")

column name    :  vat
--------------------------------
per_of_nulls   :  % 28.35
num_of_nulls   :  4513
num_of_uniques :  2
VAT deductible      10980
NaN                  4513
Price negotiable      426
Name: vat, dtype: int64


There is no relation between vat and other columns. So we can use ffill

In [58]:
df.vat.fillna(method="ffill", inplace = True)

In [59]:
df.vat.value_counts(dropna=False)

VAT deductible      15048
Price negotiable      871
Name: vat, dtype: int64

## Body_Color

In [60]:
first_looking("Body_Color")

column name    :  Body_Color
--------------------------------
per_of_nulls   :  % 3.75
num_of_nulls   :  597
num_of_uniques :  14
Black     3745
Grey      3505
White     3406
Silver    1647
Blue      1431
Red        957
NaN        597
Brown      289
Green      154
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: Body_Color, dtype: int64


In [61]:
df["Body_Color"].fillna("-", inplace = True)

In [62]:
df["Body_Color"].value_counts(dropna=False)

Black     3745
Grey      3505
White     3406
Silver    1647
Blue      1431
Red        957
-          597
Brown      289
Green      154
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: Body_Color, dtype: int64

In [63]:
df.groupby(["make_model", "body_type", 'Body_Color']).price.describe()

count      mean       std       min  \
make_model     body_type     Body_Color                                         
Audi A1        Compact       -           44.000 20291.841  3456.616 15100.000   
                             Beige        6.000 20556.500  2475.512 16240.000   
                             Black      320.000 18196.281  4206.973  9950.000   
                             Blue        96.000 19145.406  4541.863 11444.000   
                             Brown        9.000 16982.000  2964.388 11445.000   
                             Green       17.000 23558.118  3849.697 19388.000   
                             Grey       126.000 20292.627  4626.229 11800.000   
                             Red         89.000 19306.888  4779.498 11990.000   
                             Silver      42.000 17745.000  3369.815 11630.000   
                             White      274.000 19193.102  4644.280 10490.000   
                             Yellow      16.000 26756.250  3176.864 20900.000   
               Coupe         Black        2.000 14925.000  1378.858 13950.000   
               Sedans        -           29.000 20377.310  3872.915 12980.000   
                             Beige       11.000 16891.636  2557.483 14000.000   
                             Black      388.000 17423.394  4252.784 10000.000   
                             Blue       185.000 18743.989  4454.927 10800.000   
                             Brown       13.000 15037.615  1645.795 12990.000   
                             Green       15.000 22574.667  1817.576 19490.000   
                             Grey       200.000 19619.730  4795.595 11500.000   
                             Red        161.000 18666.516  4383.288 10300.000   
                             Silver      87.000 18043.414  2847.860 12750.000   
                             Violet       2.000 13700.000     0.000 13700.000   
                             White      445.000 19114.553  4441.560  8999.000   
                             Yellow      15.000 24697.800  5521.157 11950.000   
               Station wagon Black        6.000 20393.167  6628.516 15850.000   
                             Blue         5.000 18386.000  2509.767 15890.000   
                             Grey         1.000 19640.000       nan 19640.000   
                             Silver       1.000 14440.000       nan 14440.000   
                             White        8.000 15708.750  2498.383 12950.000   
               Van           Black        1.000 29000.000       nan 29000.000   
Audi A2        Off-Road      Grey         1.000 28200.000       nan 28200.000   
Audi A3        Compact       -            5.000 23464.000  4847.017 14990.000   
                             Black       59.000 16974.949  3987.638 13444.000   
                             Blue        23.000 17452.609  4825.686 13500.000   
                             Brown        2.000 15925.000  3995.153 13100.000   
                             Grey        41.000 22170.341  7749.243 12900.000   
                             Red         18.000 15355.556  2814.923 13400.000   
                             Silver       5.000 26175.800  9958.760 17990.000   
                             White       29.000 20485.828  9493.781 13790.000   
               Convertible   Black        1.000 20900.000       nan 20900.000   
                             Blue         1.000 56100.000       nan 56100.000   
                             Grey         1.000 32950.000       nan 32950.000   
                             Red          2.000 16446.500  7853.835 10893.000   
                             White        3.000 26800.000  5930.430 20000.000   
               Coupe         Silver       4.000 17745.000   962.653 16990.000   
               Sedans        -           61.000 22327.180  2814.780 12550.000   
                             Black      772.000 20609.558  5057.040 10950.000   
                             Blue       191.000 21680.775  5752.379 12500

In [64]:
df.drop("Body_Color", axis=1, inplace=True)

### Paint Type

In [65]:
first_looking("Paint_Type")

column name    :  Paint_Type
--------------------------------
per_of_nulls   :  % 36.26
num_of_nulls   :  5772
num_of_uniques :  3
Metallic       9794
NaN            5772
Uni/basic       347
Perl effect       6
Name: Paint_Type, dtype: int64


In [66]:
df["Paint_Type"].fillna("-", inplace = True)

In [67]:
df["Paint_Type"].value_counts(dropna=False)

Metallic       9794
-              5772
Uni/basic       347
Perl effect       6
Name: Paint_Type, dtype: int64

In [68]:
df.groupby(["make_model", "body_type", "age", 'Paint_Type']).price.describe()

count      mean       std  \
make_model     body_type     age   Paint_Type                                
Audi A1        Compact       0.000 -           145.000 22082.717  3341.754   
                                   Metallic    229.000 24622.258  3172.063   
                                   Uni/basic     2.000 19888.000     0.000   
                             1.000 -            56.000 17590.518  2369.313   
                                   Metallic    212.000 18861.651  2674.019   
                             2.000 -            52.000 15750.115  1218.358   
                                   Metallic    108.000 17001.352  2294.777   
                                   Uni/basic     1.000 17900.000       nan   
                             3.000 -            58.000 14007.948  1921.949   
                                   Metallic    174.000 14723.218  1881.559   
                                   Uni/basic     2.000 13200.000   424.264   
               Coupe         2.000 Metallic      1.000 15900.000       nan   
                             3.000 Metallic      1.000 13950.000       nan   
               Sedans        0.000 -           141.000 23126.972  3542.987   
                                   Metallic    269.000 24457.699  3009.668   
                                   Perl effect   1.000 28290.000       nan   
                                   Uni/basic    16.000 25071.875  3102.870   
                             1.000 -            99.000 18887.182  2797.245   
                                   Metallic    347.000 18139.533  2476.120   
                                   Uni/basic    22.000 19990.318  2440.229   
                             2.000 -           101.000 16495.723  2442.793   
                                   Metallic    150.000 16682.987  2198.296   
                                   Uni/basic    16.000 17000.562  1564.755   
                             3.000 -           152.000 14003.164  1888.368   
                                   Metallic    234.000 14757.748  1832.507   
                                   Uni/basic     3.000 15276.667  2349.901   
               Station wagon 0.000 -             1.000 28999.000       nan   
                                   Metallic      1.000 28890.000       nan   
                             1.000 Metallic     10.000 17007.000  1794.164   
                                   Uni/basic     1.000 21450.000       nan   
                             2.000 Metallic      3.000 17613.333  2953.817   
                             3.000 -             3.000 13642.667   599.950   
                                   Metallic      2.000 15431.000  1401.486   
               Van           0.000 Metallic      1.000 29000.000       nan   
Audi A2        Off-Road      1.000 Metallic      1.000 28200.000       nan   
Audi A3        Compact       0.000 -             7.000 24502.857  1950.244   
                                   Metallic     14.000 30691.643  9107.429   
                             1.000 -            18.000 21185.500  2133.694   
                                   Metallic     11.000 23414.364  6319.177   
                             2.000 -            14.000 18611.143  2186.171   
                                   Metallic     24.000 21606.458 10254.727   
                                   Uni/basic     1.000 19990.000       nan   
                             3.000 -            34.000 15172.559  1708.284   
                                   Metallic     57.000 15368.175  2519.282   
                                   Uni/basic     2.000 15700.000  3959.798   
               Convertible   1.000 -             1.000 10893.000       nan   
                                   Metallic      2.000 39050.000 24112.341   
                             2.000 -             2.000 31225.000  2439.518   
                                   Metallic      2.000 25900.000  7071.068   
                             3.000 -             1.000 20000.000       

In [69]:
df["Paint_Type"].replace("-", np.nan, inplace = True)

In [70]:
fill(df, "make_model", "body_type", "Paint_Type", "ffill")

Number of NaN :  0
------------------
Metallic       15250
Uni/basic        637
Perl effect       32
Name: Paint_Type, dtype: int64


### Type

In [71]:
first_looking("Type")

column name    :  Type
--------------------------------
per_of_nulls   :  % 0.01
num_of_nulls   :  2
num_of_uniques :  5
Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
NaN                   2
Name: Type, dtype: int64


In [72]:
df.Type.fillna("-", inplace=True)

In [73]:
df['Type'].value_counts(dropna=False)

Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
-                     2
Name: Type, dtype: int64

In [74]:
df.groupby(["Type", "make_model", "age"]).km.describe()

count       mean       std        min  \
Type           make_model     age                                             
-              Audi A3        0.000   1.000   1647.363       nan   1647.363   
                              3.000   1.000 115137.000       nan 115137.000   
Demonstration  Audi A1        0.000 110.000   3784.227  2498.415     10.000   
                              1.000  19.000   9931.158  5209.543   1050.000   
                              2.000   6.000  22963.500 17802.677  11940.000   
               Audi A3        0.000  41.000   4971.878  3092.353     50.000   
                              1.000  16.000  12801.188 10920.150   1015.000   
                              2.000   5.000  27950.000  9145.354  16000.000   
               Opel Astra     0.000 148.000   2893.376  2879.037      3.000   
                              1.000  16.000   7319.938  7499.404    100.000   
                              2.000   1.000  68500.000       nan  68500.000   
               Opel Corsa     0.000  75.000   1248.990  1687.252      3.000   
                              1.000  14.000   2915.500  5334.489     10.000   
               Opel Insignia  0.000 161.000   3936.658  3983.602      5.000   
                              1.000   6.000  13721.000 11237.453   3000.000   
                              2.000   4.000  27175.000  4251.568  23300.000   
                              3.000   1.000  56000.000       nan  56000.000   
               Renault Clio   0.000  39.000    640.310  1211.087      5.000   
                              1.000  33.000   5555.848  6384.845      8.000   
                              2.000   3.000  10014.000  9124.492     20.000   
                              3.000   1.000  35000.000       nan  35000.000   
               Renault Espace 0.000  61.000   4413.100  5077.255      5.000   
                              1.000  28.000  10118.929  7526.320     10.000   
                              2.000   8.000  24883.000 22532.018   7000.000   
Employee's car Audi A1        0.000  53.000   4506.611  4376.534      1.000   
                              1.000 131.000  12199.405  7558.408      1.000   
                              2.000   8.000  17377.750  5795.687   9767.000   
                              3.000   5.000  48103.800 30327.711  26792.000   
               Audi A2        1.000   1.000  26166.000       nan  26166.000   
               Audi A3        0.000  31.000   3869.452  5402.688     10.000   
                              1.000 157.000  18197.567  8303.327      1.000   
                              2.000  32.000  31584.750 14949.531     59.000   
                              3.000   8.000  56954.625 26796.572   7790.000   
               Opel Astra     0.000  18.000   5451.056  2971.350    431.000   
                              1.000 121.000  23088.033  8072.127     50.000   
                              2.000  12.000  52783.000 20954.520  14300.000   
                              3.000   1.000  95000.000       nan  95000.000   
               Opel Corsa     0.000  13.000   3255.566  5436.737      5.000   
                              1.000 115.000  16942.113  8966.013     10.000   
                              2.000  20.000  42133.550 10763.788  15425.000   
                              3.000   1.000 103000.000       nan 103000.000   
               Opel Insignia  0.000   6.000   7423.727 14754.766    100.000   
                              1.000 134.000  23911.075  6252.446   3158.000   
                              2.000   9.000  48409.000 45108.084  12669.000   
               Renault Clio   0.000   5.000   1459.145  1349.267      1.000   
                              1.000  79.000  24639.521 10623.131     10.000   
                              2.000  12.000  46621.083 22064.748  23000.000   
                              3.000   1.000  60892.000       nan  60892.000   
               Renault Espace 0.000   4.000   2448.681  2428.542    500.000   
         

In [75]:
cond1 = (df['make_model'] == "Audi A3") & (df["age"] == 0)
cond2 = (df['make_model'] == "Audi A3") & (df["age"] == 3)

In [76]:
df.loc[cond1,'Type'] = df.loc[cond1,'Type'].replace('-','New')
df.loc[cond2,'Type'] = df.loc[cond2,'Type'].replace('-','Used')

In [77]:
df['Type'].value_counts(dropna=False)

Used              11097
New                1651
Pre-registered     1364
Employee's car     1011
Demonstration       796
Name: Type, dtype: int64

## Inspection new

In [78]:
first_looking("Inspection_new")

column name    :  Inspection_new
--------------------------------
per_of_nulls   :  % 75.3
num_of_nulls   :  11987
num_of_uniques :  1
NaN    11987
Yes     3932
Name: Inspection_new, dtype: int64


In [79]:
df["Inspection_new"].fillna("-", inplace=True)

In [80]:
df["Inspection_new"].value_counts(dropna=False)

-      11987
Yes     3932
Name: Inspection_new, dtype: int64

In [81]:
df.groupby(["make_model", "body_type", "age", "Inspection_new"]).price.describe()

count      mean       std  \
make_model     body_type     age   Inspection_new                               
Audi A1        Compact       0.000 -              243.000 24206.370  3530.095   
                                   Yes            133.000 22542.248  3076.119   
                             1.000 -              150.000 19065.307  2836.071   
                                   Yes            118.000 17999.517  2293.776   
                             2.000 -              115.000 16425.670  1985.790   
                                   Yes             46.000 17045.652  2278.592   
                             3.000 -              128.000 14745.820  1877.293   
                                   Yes            106.000 14275.811  1923.911   
               Coupe         2.000 -                1.000 15900.000       nan   
                             3.000 -                1.000 13950.000       nan   
               Sedans        0.000 -              362.000 24130.351  3166.371   
                                   Yes             65.000 23604.262  3738.103   
                             1.000 -              323.000 18637.198  2638.317   
                                   Yes            145.000 17822.214  2368.767   
                             2.000 -              216.000 16726.495  2280.925   
                                   Yes             51.000 16227.490  2140.542   
                             3.000 -              276.000 14393.514  1892.550   
                                   Yes            113.000 14646.142  1881.027   
               Station wagon 0.000 -                2.000 28944.500    77.075   
                             1.000 -                6.000 18023.333  2482.327   
                                   Yes              5.000 16676.000  1670.054   
                             2.000 -                1.000 17440.000       nan   
                                   Yes              2.000 17700.000  4171.930   
                             3.000 -                1.000 13999.000       nan   
                                   Yes              4.000 14447.750  1456.103   
               Van           0.000 -                1.000 29000.000       nan   
Audi A2        Off-Road      1.000 -                1.000 28200.000       nan   
Audi A3        Compact       0.000 -               20.000 28815.150  8160.396   
                                   Yes              1.000 24900.000       nan   
                             1.000 -               29.000 22030.931  4270.469   
                             2.000 -               39.000 20489.769  8208.445   
                             3.000 -               90.000 15260.144  2269.181   
                                   Yes              3.000 16613.333  1572.270   
               Convertible   1.000 -                3.000 29664.333 23557.901   
                             2.000 -                4.000 28562.500  5301.159   
                             3.000 -                1.000 20000.000       nan   
               Coupe         2.000 -                1.000 19000.000       nan   
                             3.000 -                3.000 17326.667   583.124   
               Sedans        0.000 -              543.000 24980.578  2491.416   
                                   Yes            152.000 24455.250  1741.961   
                             1.000 -              635.000 22679.172  5494.472   
                                   Yes             78.000 21495.115  1289.066   
                             2.000 -              467.000 20109.559  3941.787   
                                   Yes             92.000 19205.467  1880.502   
                             3.000 -              537.000 16834.685  3078.591   
                                   Yes            117.000 16062.017  1687.635   
               Station wagon 0.000 -               85.000 24797.306  4651.885   
                                   Yes             26.000 23915.538  1589.687   
  

In [82]:
df["Inspection_new"].replace("-", "No", inplace=True)

In [83]:
df["Inspection_new"].value_counts(dropna=False)

No     11987
Yes     3932
Name: Inspection_new, dtype: int64

In [84]:
df["Inspection_new"].replace(["Yes", "No"], [1,0], inplace = True)

In [85]:
df["Inspection_new"].value_counts(dropna=False)

0    11987
1     3932
Name: Inspection_new, dtype: int64

## Upholstery_type

In [86]:
first_looking("Upholstery_type")

column name    :  Upholstery_type
--------------------------------
per_of_nulls   :  % 30.6
num_of_nulls   :  4871
num_of_uniques :  5
Cloth           8423
NaN             4871
Part leather    1499
Full leather    1009
Velour            60
alcantara         57
Name: Upholstery_type, dtype: int64


In [87]:
df["Upholstery_type"].replace(["Velour", "alcantara", "Part leather", "Full leather"], ["Cloth", "Part/Full Leather", "Part/Full Leather", "Part/Full Leather"], inplace=True)

In [88]:
df["Upholstery_type"].value_counts(dropna=False)

Cloth                8483
NaN                  4871
Part/Full Leather    2565
Name: Upholstery_type, dtype: int64

In [89]:
fill(df, "make_model", "body_type", "Upholstery_type", "ffill")

Number of NaN :  0
------------------
Cloth                12238
Part/Full Leather     3681
Name: Upholstery_type, dtype: int64


## Upholstery_color

In [90]:
df.drop("Upholstery_color", axis=1, inplace=True)

### Nr. of Doors

In [91]:
first_looking("Nr_of_Doors")

column name    :  Nr_of_Doors
--------------------------------
per_of_nulls   :  % 1.33
num_of_nulls   :  212
num_of_uniques :  6
5.000    11575
4.000     3079
3.000      832
2.000      219
nan        212
7.000        1
1.000        1
Name: Nr_of_Doors, dtype: int64


In [92]:
fill(df, "make_model", "body_type", "Nr_of_Doors", "mode")

Number of NaN :  0
------------------
5.000    11787
4.000     3079
3.000      832
2.000      219
7.000        1
1.000        1
Name: Nr_of_Doors, dtype: int64


### Nr. of Seats

In [93]:
first_looking("Nr_of_Seats")

column name    :  Nr_of_Seats
--------------------------------
per_of_nulls   :  % 6.14
num_of_nulls   :  977
num_of_uniques :  6
5.000    13336
4.000     1125
nan        977
7.000      362
2.000      116
6.000        2
3.000        1
Name: Nr_of_Seats, dtype: int64


In [94]:
fill(df, "make_model", "body_type", "Nr_of_Seats", "mode")

Number of NaN :  0
------------------
5.000    14308
4.000     1127
7.000      362
2.000      119
6.000        2
3.000        1
Name: Nr_of_Seats, dtype: int64


### Cylinders

In [95]:
first_looking("Cylinders")

column name    :  Cylinders
--------------------------------
per_of_nulls   :  % 35.68
num_of_nulls   :  5680
num_of_uniques :  7
4.000    8105
nan      5680
3.000    2104
5.000      22
6.000       3
2.000       2
8.000       2
1.000       1
Name: Cylinders, dtype: int64


In [96]:
fill(df, "make_model", "body_type", "Cylinders", "mode")

Number of NaN :  0
------------------
4.000    12926
3.000     2963
5.000       22
6.000        3
2.000        2
8.000        2
1.000        1
Name: Cylinders, dtype: int64


In [97]:
df.drop("Cylinders", axis = 1, inplace = True)

### Drive chain

In [98]:
first_looking("Drive_chain")

column name    :  Drive_chain
--------------------------------
per_of_nulls   :  % 43.08
num_of_nulls   :  6858
num_of_uniques :  3
front    8886
NaN      6858
4WD       171
rear        4
Name: Drive_chain, dtype: int64


In [99]:
df["Drive_chain"].fillna("-", inplace=True)

In [100]:
df.groupby(["make_model", "body_type", "Drive_chain"]).price.describe()

count      mean       std  \
make_model     body_type     Drive_chain                                
Audi A1        Compact       -            352.000 17620.869  4226.116   
                             4WD            2.000 14790.000  1258.650   
                             front        685.000 20008.223  4511.348   
               Coupe         -              2.000 14925.000  1378.858   
               Sedans        -            561.000 17830.440  4362.321   
                             4WD            1.000 15450.000       nan   
                             front        989.000 19133.794  4441.969   
               Station wagon -              3.000 24593.000  7537.216   
                             front         18.000 16681.111  2493.673   
               Van           front          1.000 29000.000       nan   
Audi A2        Off-Road      front          1.000 28200.000       nan   
Audi A3        Compact       -             25.000 18236.520  3427.236   
                             4WD            1.000 67600.000       nan   
                             front        156.000 18839.365  6078.743   
               Convertible   -              4.000 26112.500  6123.775   
                             4WD            1.000 56100.000       nan   
                             front          3.000 20897.667 10003.500   
               Coupe         -              1.000 16990.000       nan   
                             front          3.000 17996.667  1005.004   
               Sedans        -            891.000 20787.714  5846.561   
                             4WD           17.000 32913.765 20353.178   
                             front       1713.000 21223.413  3546.978   
               Station wagon -            149.000 20254.745  5473.077   
                             front        133.000 21165.647  3552.001   
Opel Astra     Compact       -             81.000 17993.346  6740.198   
                             front        104.000 14802.837  5080.910   
               Coupe         -              2.000 22240.000 10960.155   
               Off-Road      front          1.000 18590.000       nan   
               Sedans        -            507.000 15488.444  4123.966   
                             4WD            2.000 13075.000  3641.600   
                             front        544.000 16416.243  4600.632   
               Station wagon -            588.000 14923.395  4925.886   
                             front        697.000 16290.032  5318.146   
Opel Corsa     Compact       -            678.000 10877.917  2906.949   
                             front        641.000 12481.011  3015.968   
               Coupe         -              6.000  9198.333  1848.463   
                             front          7.000  8407.143  1342.395   
               Off-Road      -              3.000  8166.667   461.880   
               Sedans        -            359.000  9755.081  2483.224   
                             front        516.000 10519.351  2692.189   
               Transporter   -              1.000 10500.000       nan   
                             front          6.000  7175.000  2073.343   
               Van           -              2.000  7900.000     0.000   
Opel Insignia  Compact       -             17.000 28728.353 10844.301   
                             front         10.000 28355.000 12606.283   
               Coupe         -              1.000 17094.000       nan   
               Off-Road      -              1.000 33490.000       nan   
                             4WD            5.000 28232.800  8948.569   
                             front          1.000 31505.000       nan   
               Sedans        -            368.000 19794.959  6394.251   
                             4WD           26.000 34139.346  6543.818   
                             front        505.000 21675.178  7462.771   
                             rear           1.000 18990.000       nan   
               Station wagon 

In [101]:
cond = (df['make_model'] == "Renault Duster") & (df["body_type"] == "Off-Road")

In [102]:
df.loc[cond,'Drive_chain'] = df.loc[cond,'Drive_chain'].replace('-','4WD')

In [103]:
df["Drive_chain"].value_counts(dropna=False)

front    8886
-        6826
4WD       203
rear        4
Name: Drive_chain, dtype: int64

In [104]:
df["Drive_chain"] = df["Drive_chain"].replace('-', np.nan)

In [105]:
df["Drive_chain"].value_counts(dropna=False)

front    8886
NaN      6826
4WD       203
rear        4
Name: Drive_chain, dtype: int64

In [106]:
fill(df, "make_model", "body_type", "Drive_chain", "mode")

Number of NaN :  0
------------------
front    15711
4WD        204
rear         4
Name: Drive_chain, dtype: int64


### Emission Class

In [107]:
first_looking("Emission_Class")

column name    :  Emission_Class
--------------------------------
per_of_nulls   :  % 22.79
num_of_nulls   :  3628
num_of_uniques :  3
Euro 6    12173
NaN        3628
Euro 5       78
Euro 4       40
Name: Emission_Class, dtype: int64


In [108]:
df["Emission_Class"].fillna("-", inplace=True)

In [109]:
df["Emission_Class"].value_counts(dropna=False)

Euro 6    12173
-          3628
Euro 5       78
Euro 4       40
Name: Emission_Class, dtype: int64

In [110]:
df.groupby(["make_model", "age", "Fuel", "Emission_Class"]).price.describe()

count      mean       std  \
make_model     age   Fuel     Emission_Class                               
Audi A1        0.000 Benzine  -              267.000 23770.318  3136.485   
                              Euro 6         535.000 23948.533  3472.575   
                     Diesel   Euro 6           4.000 19370.000  1181.891   
               1.000 Benzine  -               99.000 19317.061  2855.338   
                              Euro 5           3.000 17862.667  1840.593   
                              Euro 6         334.000 18793.060  2829.853   
                     Diesel   -               15.000 18789.800  2474.874   
                              Euro 5           1.000 16800.000       nan   
                              Euro 6         295.000 17755.217  2063.238   
               2.000 Benzine  -               57.000 17858.070  2525.660   
                              Euro 5           1.000 10999.000       nan   
                              Euro 6         213.000 16718.394  2172.885   
                     Diesel   -               29.000 15002.207  1467.289   
                              Euro 6         132.000 16343.379  1851.918   
               3.000 Benzine  -               58.000 14752.707  2108.495   
                              Euro 4           1.000 12990.000       nan   
                              Euro 6         260.000 14495.550  2027.213   
                     Diesel   -               35.000 14612.086  1958.519   
                              Euro 5           4.000 14210.000  1478.310   
                              Euro 6         271.000 14421.815  1699.495   
Audi A2        1.000 Diesel   Euro 6           1.000 28200.000       nan   
Audi A3        0.000 Benzine  -              104.000 25614.673  3902.885   
                              Euro 6         398.000 24575.962  3290.488   
                     Diesel   -              114.000 25228.123  1824.569   
                              Euro 6         202.000 24936.322  1953.759   
                     LPG/CNG  -                2.000 24300.000     0.000   
                              Euro 6           7.000 29361.143  3573.369   
               1.000 Benzine  -               33.000 22359.303  3664.602   
                              Euro 6         119.000 25687.664 11646.909   
                     Diesel   -               31.000 21614.355  1609.743   
                              Euro 5           1.000 20500.000       nan   
                              Euro 6         590.000 21844.547  2194.097   
                     Electric Euro 6           3.000 37449.667  1361.635   
               2.000 Benzine  -               17.000 20045.235  3616.920   
                              Euro 6         125.000 20985.752  6870.602   
                     Diesel   -               65.000 18335.600  1433.653   
                              Euro 6         462.000 19844.154  3033.710   
                     LPG/CNG  Euro 6           6.000 22566.667  2001.666   
               3.000 Benzine  -               42.000 15427.881  1135.562   
                              Euro 5           1.000 14550.000       nan   
                              Euro 6          99.000 16457.121  4422.135   
                     Diesel   -               68.000 15673.750  1530.019   
                              Euro 5           5.000 16698.000  2111.521   
                              Euro 6         581.000 16544.117  2580.728   
                     Electric Euro 6           1.000 24950.000       nan   
                     LPG/CNG  -                5.000 18220.000   156.525   
                              Euro 6          16.000 18749.938  2688.004   
Opel Astra     0.000 Benzine  -              226.000 21061.487  5189.516   
                              Euro 5           1.000 17600.000       nan   
                              Euro 6         357.000 21692.941  5173.082   
                     Diesel   -               44.000 18518.455  4184.688   
                   

In [111]:
df["Emission_Class"].replace("-", np.nan, inplace=True)

In [112]:
df["Emission_Class"].value_counts(dropna=False)

Euro 6    12173
NaN        3628
Euro 5       78
Euro 4       40
Name: Emission_Class, dtype: int64

In [113]:
fill(df, "age", "Fuel", "Emission_Class", "ffill")

Number of NaN :  0
------------------
Euro 6    15769
Euro 5      100
Euro 4       50
Name: Emission_Class, dtype: int64


In [114]:
df.drop("Emission_Class", axis=1, inplace=True)

### Gears

In [115]:
first_looking("Gears")

column name    :  Gears
--------------------------------
per_of_nulls   :  % 29.6
num_of_nulls   :  4712
num_of_uniques :  10
6.000     5822
nan       4712
5.000     3239
7.000     1908
8.000      224
9.000        6
4.000        2
3.000        2
1.000        2
50.000       1
2.000        1
Name: Gears, dtype: int64


In [116]:
df["Gears"].fillna("-", inplace=True)

In [117]:
df["Gears"].value_counts(dropna=False)

6.0     5822
-       4712
5.0     3239
7.0     1908
8.0      224
9.0        6
4.0        2
3.0        2
1.0        2
50.0       1
2.0        1
Name: Gears, dtype: int64

In [118]:
df.groupby(["make_model", "body_type", "Gearing_Type", "Gears"]).price.describe()

count      mean       std  \
make_model     body_type     Gearing_Type   Gears                               
Audi A1        Compact       Automatic      5.0     3.000 22184.333  3421.302   
                                            6.0     6.000 21038.333  4282.039   
                                            7.0   199.000 22059.251  3918.116   
                                            8.0     1.000 16880.000       nan   
                                            -     253.000 21640.427  4965.500   
                             Manual         5.0   277.000 16329.469  3040.933   
                                            6.0    77.000 20538.299  2061.004   
                                            -     220.000 16756.727  3182.991   
                             Semi-automatic 7.0     3.000 24028.333  7208.440   
               Coupe         Manual         5.0     1.000 13950.000       nan   
                                            -       1.000 15900.000       nan   
               Sedans        Automatic      5.0    14.000 18078.571  1329.649   
                                            6.0    11.000 22974.364  5983.728   
                                            7.0   398.000 21320.038  4146.217   
                                            8.0     1.000 20500.000       nan   
                                            -     257.000 21100.016  4361.498   
                             Manual         5.0   513.000 15490.267  2529.015   
                                            6.0    89.000 21144.978  2273.883   
                                            7.0     1.000 15500.000       nan   
                                            -     213.000 16629.319  2916.426   
                             Semi-automatic 6.0     1.000 37900.000       nan   
                                            7.0    52.000 20432.635  5370.504   
                                            -       1.000 20900.000       nan   
               Station wagon Automatic      7.0     3.000 23193.000  5069.983   
                                            -       5.000 20970.400  4904.192   
                             Manual         5.0    10.000 15574.000  1198.621   
                                            6.0     1.000 13979.000       nan   
                                            -       2.000 14944.500  1337.139   
               Van           Automatic      6.0     1.000 29000.000       nan   
Audi A2        Off-Road      Manual         -       1.000 28200.000       nan   
Audi A3        Compact       Automatic      6.0     4.000 17820.000  1839.438   
                                            7.0    56.000 19340.982  3092.938   
                                            -      10.000 19448.400  2145.488   
                             Manual         5.0     1.000 18900.000       nan   
                                            6.0    87.000 16091.356  3171.229   
                                            7.0     1.000 15000.000       nan   
                                            -       9.000 17373.333  3100.387   
                             Semi-automatic 6.0     1.000 24950.000       nan   
                                            7.0    13.000 38341.308  9924.338   
               Convertible   Automatic      7.0     3.000 17931.000  6119.852   
                             Semi-automatic 6.0     2.000 25450.000  7707.464   
                                            7.0     3.000 39516.667 14464.814   
               Coupe         Automatic      7.0     4.000 17745.000   962.653   
               Sedans        Automatic      6.0    72.000 18276.889  2859.820   
                                            7.0   790.000 21967.932  3273.470   
                                            -     312.000 21566.500  3834.391   
                             Manual         5.0     1.000 12949.000       nan   
                                            6.0   935.000 19965.568  3279.827   
  

In [119]:
df["Gears"].replace([1,2,3,4,9,50,"-"], np.nan, inplace=True)

In [120]:
df["Gears"].value_counts(dropna=False)

6.000    5822
nan      4726
5.000    3239
7.000    1908
8.000     224
Name: Gears, dtype: int64

In [121]:
for group1 in list(df["make_model"].unique()):
    for group2 in list(df["body_type"].unique()):
        for group3 in list(df["Gearing_Type"].unique()):
            cond1 = df["make_model"]==group1
            cond2 = (df["make_model"]==group1) & (df["body_type"]==group2)
            cond3 = (df["make_model"]==group1) & (df["body_type"]==group2) & (df["Gearing_Type"]==group3)
            mode1 = list(df[cond1]["Gears"].mode())
            mode2 = list(df[cond2]["Gears"].mode())
            mode3 = list(df[cond3]["Gears"].mode())
            if mode3 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond3]["Gears"].mode()[0])
            elif mode2 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond2]["Gears"].mode()[0])
            elif mode1 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond1]["Gears"].mode()[0])
            else:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df["Gears"].mode()[0])

In [122]:
df["Gears"].value_counts(dropna=False)

6.000    8626
5.000    4258
7.000    2810
8.000     225
Name: Gears, dtype: int64

### hp_kW

In [123]:
first_looking("hp_kW")

column name    :  hp_kW
--------------------------------
per_of_nulls   :  % 0.55
num_of_nulls   :  88
num_of_uniques :  80
85.000     2542
66.000     2122
81.000     1402
100.000    1308
110.000    1112
70.000      888
125.000     707
51.000      695
55.000      569
118.000     516
92.000      466
121.000     392
147.000     380
77.000      345
56.000      286
54.000      276
103.000     253
87.000      232
165.000     194
88.000      177
60.000      160
162.000      98
nan          88
74.000       81
96.000       72
71.000       59
101.000      47
67.000       40
154.000      39
122.000      35
119.000      30
164.000      27
135.000      24
82.000       22
52.000       22
1.000        20
78.000       20
294.000      18
146.000      18
141.000      16
57.000       10
104.000       8
120.000       8
112.000       7
191.000       7
117.000       6
155.000       6
184.000       5
65.000        4
90.000        4
76.000        4
98.000        3
168.000       3
93.000        3
149.000     

In [124]:
df["hp_kW"].fillna("-", inplace=True)

In [125]:
df.groupby(["make_model", "body_type","hp_kW"]).price.describe()

count      mean       std       min  \
make_model     body_type     hp_kW                                          
Audi A1        Compact       60.0    58.000 15189.828  1663.611 10900.000   
                             66.0   162.000 15398.210  1934.301 10490.000   
                             70.0   332.000 17983.262  3315.814  9950.000   
                             71.0    32.000 20831.594  2410.840 15890.000   
                             85.0   330.000 22604.103  4572.568 11100.000   
                             86.0     1.000 14295.000       nan 14295.000   
                             92.0    90.000 18029.056  3027.431 12550.000   
                             93.0     2.000 21447.500   774.282 20900.000   
                             110.0   20.000 23620.450  3777.344 15490.000   
                             141.0    2.000 22495.000   841.457 21900.000   
                             147.0    8.000 28797.125   355.356 28200.000   
                             -        2.000 19125.000  5126.524 15500.000   
               Coupe         70.0     1.000 13950.000       nan 13950.000   
                             85.0     1.000 15900.000       nan 15900.000   
               Sedans        60.0    95.000 15218.168  1624.267 11900.000   
                             66.0   296.000 15922.797  2331.608 10800.000   
                             67.0    12.000 15432.417  2778.191 11990.000   
                             70.0   455.000 17282.068  3210.456  8999.000   
                             71.0    24.000 19177.250  2539.495 14390.000   
                             85.0   555.000 21609.523  4822.007 10000.000   
                             87.0     4.000 14865.000   750.000 14490.000   
                             92.0    79.000 19078.975  2790.209 13000.000   
                             93.0     1.000 17900.000       nan 17900.000   
                             110.0    9.000 22591.333  4188.635 16999.000   
                             112.0    1.000 18399.000       nan 18399.000   
                             140.0    1.000 23490.000       nan 23490.000   
                             141.0   14.000 21232.000  2597.466 14500.000   
                             147.0    5.000 30729.600  4008.782 28880.000   
               Station wagon 60.0     3.000 15106.667  1890.882 12950.000   
                             66.0     7.000 16018.429  1779.762 13999.000   
                             70.0     3.000 17420.000  3283.474 14440.000   
                             85.0     4.000 22702.750  7207.560 16422.000   
                             92.0     3.000 17623.000  3738.860 13979.000   
                             110.0    1.000 20650.000       nan 20650.000   
               Van           147.0    1.000 29000.000       nan 29000.000   
Audi A2        Off-Road      85.0     1.000 28200.000       nan 28200.000   
Audi A3        Compact       81.0    50.000 17147.920  2283.775 12900.000   
                             82.0    11.000 17042.091  1353.481 14450.000   
                             85.0    53.000 22656.717  6328.251 13790.000   
                             87.0     3.000 22195.000   349.887 21990.000   
                             92.0    43.000 13824.884   984.708 13100.000   
                             110.0   15.000 25249.333  7908.351 13990.000   
                             112.0    1.000 17894.000       nan 17894.000   
                             135.0    3.000 15950.000     0.000 15950.000   
                             150.0    1.000 24950.000       nan 24950.000   
                             294.0    1.000 67600.000       nan 67600.000   
                             -        1.000 18700.000       nan 18700.000   
               Convertible   81.0     1.000 20900.000       nan 20900.000   
                             110.0    5.000 24848.600  9248.721 10893.000   
                             135.0    1.000 22000.000       nan 22000.000   
                             228.0

In [126]:
df["hp_kW"].replace("-", np.nan, inplace=True)

In [127]:
fill(df, "make_model", "body_type", "hp_kW", "mode")

Number of NaN :  0
------------------
85.000     2543
66.000     2124
81.000     1403
100.000    1314
110.000    1113
70.000      890
125.000     711
51.000      696
55.000      589
118.000     550
92.000      466
121.000     392
147.000     380
77.000      353
56.000      294
54.000      276
103.000     253
87.000      232
165.000     194
88.000      177
60.000      160
162.000      98
74.000       81
96.000       72
71.000       59
101.000      47
67.000       40
154.000      39
122.000      35
119.000      30
164.000      27
135.000      24
82.000       22
52.000       22
1.000        20
78.000       20
146.000      18
294.000      18
141.000      16
57.000       10
104.000       8
120.000       8
191.000       7
112.000       7
117.000       6
155.000       6
184.000       5
76.000        4
90.000        4
65.000        4
98.000        3
168.000       3
149.000       3
80.000        3
93.000        3
150.000       2
143.000       2
167.000       2
140.000       2
127.000       2
22

## Displacement_cc

In [128]:
first_looking("Displacement_cc")

column name    :  Displacement_cc
--------------------------------
per_of_nulls   :  % 3.12
num_of_nulls   :  496
num_of_uniques :  77
1598.000     4761
999.000      2438
1398.000     1314
1399.000      749
1229.000      677
1956.000      670
1461.000      595
1490.000      559
nan           496
1422.000      467
1197.000      353
898.000       351
1395.000      320
1968.000      301
1149.000      288
1618.000      212
1798.000      210
1498.000      196
1600.000      130
1248.000      110
1997.000      103
1364.000      102
1400.000       90
998.000        72
1500.000       50
2000.000       46
1000.000       40
1.000          36
1998.000       25
2480.000       20
1984.000       18
1200.000       18
899.000        11
1397.000       11
160.000         6
1499.000        5
929.000         5
900.000         4
997.000         4
1596.000        4
139.000         4
1396.000        3
1599.000        3
1199.000        3
995.000         2
1495.000        2
2.000           2
1300.000        2
1

In [129]:
df["Displacement_cc"].fillna("-", inplace=True)

In [130]:
df.groupby(["make_model", "body_type","Displacement_cc"]).price.describe()

count      mean       std  \
make_model     body_type     Displacement_cc                                
Audi A1        Compact       929.0              5.000 16364.000  1504.819   
                             995.0              2.000 21990.000  1414.214   
                             999.0            663.000 20294.101  4570.798   
                             1000.0             7.000 19498.429  5396.196   
                             1395.0            98.000 18246.622  3084.081   
                             1422.0           158.000 15396.424  1943.650   
                             1498.0            14.000 24982.143  3186.063   
                             1596.0             1.000 15585.000       nan   
                             1598.0            68.000 16568.309  3059.451   
                             1600.0             1.000 16800.000       nan   
                             1798.0             2.000 22495.000   841.457   
                             1984.0             8.000 28797.125   355.356   
                             -                 12.000 17960.833  5247.776   
               Coupe         999.0              1.000 13950.000       nan   
                             1598.0             1.000 15900.000       nan   
               Sedans        998.0              2.000 20990.000     0.000   
                             999.0            882.000 19883.027  4867.721   
                             1000.0             7.000 26414.286  3161.006   
                             1197.0             1.000 15980.000       nan   
                             1395.0            85.000 18973.647  2474.698   
                             1422.0           302.000 15917.010  2361.308   
                             1498.0             2.000 28950.000    70.711   
                             1596.0             1.000 17700.000       nan   
                             1598.0           233.000 16802.601  2465.628   
                             1600.0             1.000 13290.000       nan   
                             1798.0            14.000 21232.000  2597.466   
                             1984.0             5.000 30729.600  4008.782   
                             -                 16.000 17966.875  3330.743   
               Station wagon 999.0              7.000 18067.143  5375.828   
                             1395.0             4.000 18379.750  3407.355   
                             1422.0             7.000 16018.429  1779.762   
                             1598.0             2.000 16461.000    55.154   
                             -                  1.000 28999.000       nan   
               Van           1984.0             1.000 29000.000       nan   
Audi A2        Off-Road      1598.0             1.000 28200.000       nan   
Audi A3        Compact       999.0             16.000 27854.812  9229.154   
                             1197.0             1.000 18495.000       nan   
                             1395.0            48.000 14657.583  3256.004   
                             1400.0             1.000 24950.000       nan   
                             1495.0             2.000 27611.000  3307.846   
                             1498.0             1.000 24490.000       nan   
                             1499.0             2.000 39995.000  2121.320   
                             1598.0            97.000 18544.206  2763.070   
                             1600.0             1.000 17900.000       nan   
                             1968.0            10.000 19208.800  4834.476   
                             2480.0             1.000 67600.000       nan   
                             -                  2.000 16845.000  2623.366   
               Convertible   1395.0             1.000 32950.000       nan   
                             1498.0             1.000 10893.000       nan   
                             1598.0             1.000 20900.000       nan   
                             1968.0         

In [131]:
df["Displacement_cc"].replace("-", np.nan, inplace=True)

In [132]:
fill(df, "make_model", "body_type", "Displacement_cc", "mode")

Number of NaN :  0
------------------
1598.000     5044
999.000      2467
1398.000     1387
1399.000      768
1229.000      678
1956.000      670
1461.000      667
1490.000      559
1422.000      467
1197.000      372
898.000       351
1395.000      320
1968.000      301
1149.000      288
1618.000      212
1798.000      210
1498.000      196
1600.000      130
1248.000      110
1997.000      103
1364.000      102
1400.000       90
998.000        72
1500.000       50
2000.000       46
1000.000       40
1.000          36
1998.000       25
2480.000       20
1200.000       18
1984.000       18
899.000        11
1397.000       11
160.000         6
929.000         5
1499.000        5
900.000         4
997.000         4
1596.000        4
139.000         4
1396.000        3
1599.000        3
1199.000        3
1589.000        2
1300.000        2
1495.000        2
2.000           2
995.000         2
1533.000        1
996.000         1
1239.000        1
1369.000        1
1686.000        1
1100.000

## Weight_kg

In [133]:
first_looking("Weight_kg")

column name    :  Weight_kg
--------------------------------
per_of_nulls   :  % 43.81
num_of_nulls   :  6974
num_of_uniques :  434
nan         6974
1163.000     574
1360.000     356
1165.000     301
1335.000     242
1135.000     213
1199.000     205
1734.000     170
1180.000     168
1503.000     165
1350.000     155
1355.000     135
1260.000     127
1275.000     112
1278.000     110
1487.000     109
1425.000     109
1255.000     108
1200.000     107
1273.000     103
1522.000     103
1280.000     102
1403.000      91
1120.000      90
1659.000      89
1195.000      89
1701.000      87
1250.000      84
1441.000      82
1285.000      80
1308.000      80
1110.000      75
1613.000      75
1279.000      72
1364.000      70
1345.000      67
1733.000      65
1141.000      64
1685.000      64
1325.000      64
1071.000      64
1230.000      63
1845.000      56
1090.000      54
1052.000      53
1664.000      52
1154.000      52
1513.000      51
1065.000      50
1237.000      49
1205.000      46
1

In [134]:
df["Weight_kg"].fillna("-", inplace=True)

In [135]:
df.groupby(["make_model", "body_type","Weight_kg"]).price.describe()

count      mean       std       min  \
make_model     body_type     Weight_kg                                         
Audi A1        Compact       102.0       1.000 19229.000       nan 19229.000   
                             1010.0      2.000 15450.000   707.107 14950.000   
                             1035.0      6.000 16796.667  2617.867 14390.000   
                             1040.0      2.000 20424.500  2933.786 18350.000   
                             1065.0     36.000 20971.778  1982.554 15500.000   
                             1090.0      9.000 21059.222  2945.233 15998.000   
                             1100.0      2.000 27525.000  1732.412 26300.000   
                             1105.0     12.000 13629.167   228.093 13400.000   
                             1110.0     22.000 14322.000  1738.097  9950.000   
                             1115.0      3.000 20833.333  1950.855 18850.000   
                             1125.0      1.000 13945.000       nan 13945.000   
                             1133.0      3.000 21223.333   251.661 20990.000   
                             1134.0      6.000 16258.333   988.138 14900.000   
                             1135.0     53.000 16554.679  2170.539 12980.000   
                             1138.0      1.000 22140.000       nan 22140.000   
                             1140.0      1.000 21940.000       nan 21940.000   
                             1147.0      1.000 22990.000       nan 22990.000   
                             1150.0      1.000 22980.000       nan 22980.000   
                             1155.0      5.000 18533.000  3476.373 13480.000   
                             1165.0     77.000 20264.260  2560.317 12900.000   
                             1180.0     86.000 20753.616  2638.047 13475.000   
                             1195.0     40.000 15930.725  1308.219 11290.000   
                             1200.0     65.000 26139.554  2769.603 18730.000   
                             1205.0      5.000 15098.800  1223.636 13750.000   
                             1215.0      9.000 20182.556  2365.729 16879.000   
                             1220.0      2.000 18090.000  3733.524 15450.000   
                             1225.0      4.000 15887.750  1550.406 14391.000   
                             1230.0      3.000 23250.000  2655.654 20500.000   
                             1235.0      2.000 16590.000   141.421 16490.000   
                             1240.0      7.000 25600.714  3398.826 20980.000   
                             1250.0     17.000 16218.588  3691.278 11100.000   
                             1275.0      5.000 20453.800  4993.467 15889.000   
                             1280.0      1.000 25490.000       nan 25490.000   
                             1285.0      3.000 15516.667  1590.985 13680.000   
                             1335.0      3.000 28832.333   237.563 28560.000   
                             1345.0      3.000 16106.667   317.857 15880.000   
                             1485.0      5.000 15320.000   408.656 15000.000   
                             1500.0      1.000 13500.000       nan 13500.000   
                             1580.0      1.000 14280.000       nan 14280.000   
                             1600.0      4.000 15550.000   700.000 14500.000   
                             1625.0      1.000 15990.000       nan 15990.000   
                             1675.0      1.000 11470.000       nan 11470.000   
                             1792.0      1.000 13990.000       nan 13990.000   
                             -         526.000 18738.951  4732.907 10490.000   
               Coupe         1010.0      1.000 13950.000       nan 13950.000   
                             -           1.000 15900.000       nan 15900.000   
               Sedans        1035.0      5.000 16348.000   560.598 15450.000   
                             1060.0      2.000 14940.000  1484.924 13890.000   
                             1065.0      

In [136]:
df["Weight_kg"].replace("-", np.nan, inplace=True)

In [137]:
fill(df, "make_model", "body_type", "Weight_kg", "mode")

Number of NaN :  0
------------------
1163.000    1582
1360.000    1419
1487.000     966
1135.000     837
1425.000     744
1180.000     694
1273.000     656
1165.000     603
1503.000     561
1734.000     556
1087.000     291
1335.000     242
1365.000     211
1199.000     205
1350.000     156
1119.000     153
1355.000     136
1280.000     127
1260.000     127
1275.000     112
1278.000     110
1255.000     108
1200.000     107
1522.000     103
1659.000     102
1195.000      96
1120.000      93
1403.000      91
1701.000      87
1250.000      85
1685.000      83
1441.000      82
1285.000      80
1308.000      80
1613.000      75
1110.000      75
1279.000      72
1364.000      70
1345.000      67
1733.000      65
1141.000      64
1071.000      64
1325.000      64
1209.000      64
1230.000      63
1845.000      56
1090.000      54
1052.000      53
1154.000      52
1664.000      52
1513.000      51
1065.000      50
1237.000      49
1440.000      46
1205.000      46
1088.000      46
1265.000  

## CO2 Emission

In [138]:
first_looking("CO2_Emission")

column name    :  CO2_Emission
--------------------------------
per_of_nulls   :  % 15.3
num_of_nulls   :  2436
num_of_uniques :  120
nan        2436
120.000     740
99.000      545
97.000      537
104.000     501
10.000      477
103.000     445
114.000     382
124.000     372
107.000     362
108.000     362
119.000     361
106.000     349
128.000     329
126.000     282
85.000      275
118.000     270
110.000     266
127.000     257
117.000     254
111.000     237
113.000     235
109.000     234
139.000     224
140.000     218
129.000     213
135.000     202
105.000     202
9.000       198
130.000     180
123.000     178
150.000     174
143.000     171
11.000      171
95.000      161
116.000     157
141.000     156
98.000      151
133.000     145
136.000     145
137.000     133
125.000     132
134.000     130
145.000     126
149.000     117
153.000     113
147.000     109
101.000     105
13.000      100
115.000      86
1.000        84
121.000      82
138.000      75
93.000       66
14

In [139]:
df["CO2_Emission"].fillna("-", inplace=True)

In [140]:
df.groupby(["make_model", "body_type","CO2_Emission"]).price.describe()

count      mean       std  \
make_model     body_type     CO2_Emission                                     
Audi A1        Compact       1.0                  1.000 20881.000       nan   
                             9.0                  1.000 12479.000       nan   
                             10.0               147.000 16946.871  2850.063   
                             11.0                25.000 19960.360  2676.262   
                             14.0                 2.000 28750.000   268.701   
                             90.0                 1.000 12880.000       nan   
                             91.0                 5.000 13038.000  1108.927   
                             94.0                 5.000 13960.000  1232.071   
                             97.0               113.000 15449.593  2445.755   
                             98.0                33.000 16574.364  2637.087   
                             99.0                19.000 16043.947  2257.403   
                             100.0                4.000 16127.500  1583.401   
                             103.0               30.000 21233.033  1794.551   
                             104.0              126.000 17411.413  3149.373   
                             105.0               14.000 22234.857   812.264   
                             106.0               43.000 19831.791  3335.793   
                             107.0               13.000 16527.692  1620.412   
                             108.0               42.000 23587.095  3375.693   
                             109.0               34.000 17029.324  2514.314   
                             110.0               51.000 26340.373  2434.738   
                             111.0              138.000 23765.283  3477.921   
                             113.0                5.000 20388.000  2796.188   
                             114.0                1.000 15490.000       nan   
                             115.0                7.000 24067.857  6950.650   
                             116.0               20.000 23580.950  2813.103   
                             117.0                8.000 19094.875  2942.954   
                             118.0                2.000 13615.000    91.924   
                             119.0                1.000 12550.000       nan   
                             120.0               27.000 15850.000     0.000   
                             121.0                2.000 18710.000  3224.407   
                             123.0               17.000 19308.647  3816.070   
                             126.0                1.000 19999.000       nan   
                             127.0                2.000 19888.000     0.000   
                             134.0                2.000 22495.000   841.457   
                             136.0                2.000 29088.000   128.693   
                             137.0                2.000 28660.000   282.843   
                             331.0                1.000 12990.000       nan   
                             -                   92.000 18124.228  4461.867   
               Coupe         97.0                 1.000 13950.000       nan   
                             -                    1.000 15900.000       nan   
               Sedans        9.0                 42.000 15764.548  1578.276   
                             10.0               198.000 17497.909  2518.189   
                             11.0                42.000 19247.595  2058.248   
                             89.0                 4.000 16572.500   288.141   
                             90.0                 3.000 14396.333   532.382   
                             91.0                14.000 14228.500  1438.927   
                             94.0                12.000 15710.000  2522.329   
                             95.0                 2.000 13070.000  2305.168   
                             97.0               315.000 14905.857  1996.270   
                     

In [141]:
df["CO2_Emission"].replace("-", np.nan, inplace=True)

In [142]:
fill(df, "make_model", "body_type", "CO2_Emission", "median")

Number of NaN :  0
------------------
120.000    836
104.000    679
106.000    674
114.000    556
99.000     546
97.000     539
139.000    523
124.000    516
10.000     477
117.000    460
103.000    445
123.000    410
98.000     366
107.000    362
108.000    362
119.000    361
129.000    333
133.000    330
128.000    329
118.000    314
110.000    289
126.000    284
85.000     275
127.000    257
111.000    237
113.000    235
109.000    234
140.000    221
105.000    202
135.000    202
9.000      198
130.000    189
150.000    174
11.000     171
143.000    171
95.000     161
116.000    157
141.000    156
118.500    147
136.000    145
137.000    135
125.000    133
134.000    130
145.000    126
149.000    117
153.000    113
147.000    109
101.000    105
13.000     100
115.000     86
1.000       84
121.000     82
138.000     75
93.000      66
14.000      59
168.000     58
90.000      54
131.000     48
148.000     48
144.000     40
154.000     40
94.000      37
146.000     36
100.000     36
15

In [143]:
df.drop("CO2_Emission", axis=1, inplace=True)

### Comfort_Convenience

In [144]:
first_looking("Comfort_Convenience")

column name    :  Comfort_Convenience
--------------------------------
per_of_nulls   :  % 5.78
num_of_nulls   :  920
num_of_uniques :  6198
NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         920
Air conditioning,Electrical side mirrors,Hill Holder,Power windows                                                                                                                                                                                                                                                                                                         

In [145]:
fill(df, "make_model", "body_type", "Comfort_Convenience", "mode")

Number of NaN :  0
------------------
Air conditioning,Electrical side mirrors,Hill Holder,Power windows                                                                                                                                                                                                                                                                                                                                                                                                                             388
Air conditioning,Armrest,Automatic climate control,Cruise control,Electrical side mirrors,Leather steering wheel,Light sensor,Lumbar support,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors front,Parking assist system sensors rear,Power windows,Rain sensor,Seat heating,Start-stop system                                                                                                                                             

### Entertainment_Media

In [146]:
first_looking("Entertainment_Media")

column name    :  Entertainment_Media
--------------------------------
per_of_nulls   :  % 8.63
num_of_nulls   :  1374
num_of_uniques :  346
NaN                                                                                                               1374
Bluetooth,Hands-free equipment,On-board computer,Radio,USB                                                        1282
Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB                                                     982
Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB                                           783
On-board computer,Radio                                                                                            487
Radio                                                                                                              477
Bluetooth,CD player,Hands-free equipment,On-board computer,Radio,USB                                               465
On-board computer         

In [147]:
fill(df, "make_model", "body_type", "Entertainment_Media", "mode")

Number of NaN :  0
------------------
Bluetooth,Hands-free equipment,On-board computer,Radio,USB                                                        1738
Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB                                                    1134
Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB                                          1010
On-board computer                                                                                                  615
Radio                                                                                                              558
Bluetooth,Hands-free equipment,On-board computer,Radio                                                             515
On-board computer,Radio                                                                                            487
Bluetooth,CD player,Hands-free equipment,On-board computer,Radio,USB                                               466
Bluetooth,

### Extras

In [148]:
first_looking("Extras")

column name    :  Extras
--------------------------------
per_of_nulls   :  % 18.61
num_of_nulls   :  2962
num_of_uniques :  659
Alloy wheels                                                                                                                                   3245
NaN                                                                                                                                            2962
Alloy wheels,Touch screen                                                                                                                       697
Alloy wheels,Voice Control                                                                                                                      577
Alloy wheels,Touch screen,Voice Control                                                                                                         541
Alloy wheels,Roof rack                                                                                                             

In [149]:
fill(df, "make_model", "body_type", "Extras", "mode")

Number of NaN :  0
------------------
Alloy wheels                                                                                                                                   5786
Alloy wheels,Touch screen                                                                                                                       697
Roof rack                                                                                                                                       596
Alloy wheels,Voice Control                                                                                                                      582
Alloy wheels,Touch screen,Voice Control                                                                                                         544
Alloy wheels,Roof rack                                                                                                                          529
Alloy wheels,Sport seats                                                  

### Safety_Security

In [150]:
first_looking("Safety_Security")

column name    :  Safety_Security
--------------------------------
per_of_nulls   :  % 6.17
num_of_nulls   :  982
num_of_uniques :  4443
NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                             982
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                                                                                                               

In [151]:
fill(df, "make_model", "body_type", "Safety_Security", "mode")

Number of NaN :  0
------------------
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                              729
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                                         480
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,LED Daytime Running Lights,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                   373
ABS,Cent

## cons_comb

In [152]:
first_looking("cons_comb")

column name    :  cons_comb
--------------------------------
per_of_nulls   :  % 12.77
num_of_nulls   :  2033
num_of_uniques :  72
nan       2033
5.400      770
3.900      733
4.000      713
5.100      657
4.400      623
5.600      618
4.700      602
3.800      585
4.800      546
5.000      545
4.500      523
5.200      454
4.200      435
4.600      426
4.900      393
5.500      380
5.300      380
5.900      369
3.700      369
5.700      342
4.100      342
6.000      331
3.300      307
4.300      307
3.500      288
6.200      216
3.600      194
6.300      181
6.100      175
5.800      164
6.600      148
6.800      136
3.400      106
6.400       75
3.000       69
7.400       66
6.700       43
6.500       43
7.100       38
10.000      34
6.900       27
3.200       25
8.300       20
7.600       14
7.000       10
3.100        7
7.200        6
7.800        6
8.000        5
8.600        4
51.000       4
8.700        3
7.900        3
1.600        3
38.000       2
40.000       2
7.300        2

In [153]:
cons_comb = (df["cons_country"] + df["cons_city"])/2

In [154]:
df["cons_comb"] = df["cons_comb"].fillna(cons_comb)

In [155]:
df["cons_comb"].value_counts(dropna=False)

nan       1925
5.400      770
3.900      733
4.000      713
5.100      657
4.400      623
5.600      618
4.700      607
3.800      585
4.800      546
5.000      545
4.500      524
5.200      454
4.200      435
4.600      426
4.900      393
5.300      380
5.500      380
3.700      369
5.900      369
5.700      342
4.100      342
6.000      331
4.300      307
3.300      307
3.500      288
6.200      216
3.600      194
6.300      181
6.100      175
5.800      165
6.600      148
6.800      136
3.400      125
6.400       75
3.000       69
7.400       66
6.500       43
6.700       43
7.100       38
3.600       37
10.000      34
6.900       27
3.200       25
8.300       20
3.650       20
5.150       19
7.600       14
7.000       10
3.100        7
7.200        6
7.800        6
8.000        5
51.000       4
8.600        4
3.950        3
1.600        3
8.700        3
7.900        3
40.000       2
8.100        2
38.000       2
5.450        2
7.300        2
0.000        2
1.200        1
11.000    

In [156]:
df["cons_comb"].fillna("-", inplace=True)

In [157]:
df.groupby(["make_model", "body_type","cons_comb"]).price.describe()

count      mean       std  \
make_model     body_type     cons_comb                                        
Audi A1        Compact       3.0                  4.000 14370.250  4481.393   
                             3.4                  6.000 12755.000   900.750   
                             3.5                  3.000 14613.000  1848.098   
                             3.6                  5.000 13960.000  1232.071   
                             3.7                 50.000 15902.200  2960.324   
                             3.8                 18.000 16158.056  2265.763   
                             3.9                 41.000 15771.902  1567.307   
                             4.0                 79.000 16795.671  2620.874   
                             4.1                 38.000 16540.737  1754.464   
                             4.2                 87.000 15631.126  2420.347   
                             4.3                 17.000 16628.706  2467.669   
                             4.4                193.000 17618.922  3250.845   
                             4.5                 11.000 21482.909  1918.658   
                             4.6                 52.000 20729.962  2195.464   
                             4.7                 33.000 22102.667  1889.931   
                             4.8                 93.000 25162.548  3080.468   
                             4.9                176.000 23133.290  3723.040   
                             5.0                  9.000 21739.444  2364.487   
                             5.1                 50.000 18187.280  4445.452   
                             5.2                 18.000 19429.833  3737.664   
                             5.6                  2.000 19888.000     0.000   
                             5.8                  2.000 22495.000   841.457   
                             6.0                  8.000 28797.125   355.356   
                             13.8                 1.000 12990.000       nan   
                             -                   43.000 16765.791  4426.026   
               Coupe         4.2                  1.000 13950.000       nan   
                             -                    1.000 15900.000       nan   
               Sedans        3.0                  1.000 15450.000       nan   
                             3.4                 20.000 14829.400  1517.956   
                             3.5                 48.000 15679.125  1535.780   
                             3.6                 13.000 15590.000  2503.514   
                             3.7                140.000 15242.100  2130.600   
                             3.8                 95.000 16135.463  2095.939   
                             3.9                 93.000 16511.710  1919.916   
                             4.0                 81.000 19200.000  2282.083   
                             4.1                 15.000 16189.333  2096.758   
                             4.2                220.000 14813.968  1838.279   
                             4.3                 43.000 16173.558  1713.437   
                             4.4                156.000 17887.404  2900.445   
                             4.5                  8.000 19561.000  1725.197   
                             4.55                 1.000 11990.000       nan   
                             4.6                 61.000 20956.197  1483.175   
                             4.7                  9.000 20541.333  2427.125   
                             4.8                257.000 25246.179  2742.582   
                             4.9                 84.000 21241.690  3559.583   
                             5.0                  7.000 23258.571  5778.473   
                             5.1                 27.000 16992.074  2964.090   
                             5.2                  5.000 20150.200  2021.626   
                             5.3                  5.000 20304.600  1130.643   
                     

In [158]:
df["cons_comb"].replace([0.0, 1.0, 1.2, 1.6, 10, 11, 13.8, 32.0, 33.0, 38.0, 40.0, 43.0, 46.0, 50.0, 51.0, 54.0, 55.0, "-"], np.nan, inplace=True)

In [159]:
df["cons_comb"].value_counts(dropna=False)

nan      1983
5.400     770
3.900     733
4.000     713
5.100     657
4.400     623
5.600     618
4.700     607
3.800     585
4.800     546
5.000     545
4.500     524
5.200     454
4.200     435
4.600     426
4.900     393
5.500     380
5.300     380
3.700     369
5.900     369
4.100     342
5.700     342
6.000     331
4.300     307
3.300     307
3.500     288
6.200     216
3.600     194
6.300     181
6.100     175
5.800     165
6.600     148
6.800     136
3.400     125
6.400      75
3.000      69
7.400      66
6.500      43
6.700      43
7.100      38
3.600      37
6.900      27
3.200      25
8.300      20
3.650      20
5.150      19
7.600      14
7.000      10
3.100       7
7.200       6
7.800       6
8.000       5
8.600       4
3.950       3
8.700       3
7.900       3
8.100       2
7.300       2
5.450       2
4.550       1
7.500       1
9.100       1
Name: cons_comb, dtype: int64

In [160]:
fill(df, "make_model", "body_type", "cons_comb", "median")

Number of NaN :  0
------------------
4.000    1020
5.600     965
5.400     900
5.000     810
4.700     766
3.900     733
4.500     731
4.400     668
5.100     657
3.800     585
4.800     580
4.200     579
4.900     535
5.300     530
5.200     454
4.600     426
5.500     389
3.700     369
5.900     369
4.100     362
5.700     342
6.000     331
4.300     307
3.300     307
3.500     288
6.200     219
3.600     194
6.300     181
6.100     175
5.800     165
6.600     148
6.800     136
3.400     125
6.400      75
3.000      69
7.400      66
6.700      43
6.500      43
7.100      38
3.600      37
6.900      27
3.200      25
8.300      20
3.650      20
3.450      19
5.150      19
7.600      14
7.000      10
3.100       7
7.200       6
7.800       6
8.000       5
8.600       4
8.700       3
3.950       3
7.900       3
8.100       2
7.300       2
5.450       2
4.550       2
7.500       1
4.300       1
9.100       1
Name: cons_comb, dtype: int64


## cons_country

In [161]:
df.drop("cons_country", axis = 1, inplace = True)

## cons_city 

In [162]:
df.drop("cons_city", axis = 1, inplace = True)

## End of this phase

In [163]:
df.shape

(15919, 25)

In [164]:
df.isnull().sum()/df.shape[0]*100

make_model            0.000
body_type             0.000
price                 0.000
vat                   0.000
km                    0.000
Type                  0.000
Fuel                  0.000
Gears                 0.000
Comfort_Convenience   0.000
Entertainment_Media   0.000
Extras                0.000
Safety_Security       0.000
age                   0.000
Previous_Owners       0.000
hp_kW                 0.000
Inspection_new        0.000
Paint_Type            0.000
Upholstery_type       0.000
Nr_of_Doors           0.000
Nr_of_Seats           0.000
Gearing_Type          0.000
Displacement_cc       0.000
Weight_kg             0.000
Drive_chain           0.000
cons_comb             0.000
dtype: float64

In [165]:
df.to_csv("filled_scout.csv", index=False)